# TimeZones in Python

In [1]:
# load libraries
import datetime
import pytz
import dateutil
import numpy
import pandas

In [2]:
# change working path
import os
os.chdir('../')

In [3]:
datetime.datetime.now()

datetime.datetime(2020, 11, 6, 17, 51, 15, 238145)

In [4]:
datetime.datetime.utcnow()

datetime.datetime(2020, 11, 6, 23, 51, 15, 652379)

In [5]:
# datetime with timezone added
datetime.datetime.now(datetime.timezone.utc)

datetime.datetime(2020, 11, 6, 23, 51, 16, 9848, tzinfo=datetime.timezone.utc)

Notice that if we do pass in a time zone, we will get the correct information, but this
is not the default behavior. To work with time zones in Python, we create a time zone
object, such western as for the US Pacific time zone:

In [6]:
western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

We can then use these objects to localize a time zone as follows:


In [7]:
loc_td = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))

In [8]:
loc_td

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

Note, however, that passing the time zone directly into the datetime constructor will
often not produce the result we were expecting

In [9]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_td.astimezone(london_tz)

In [10]:
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [11]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 16, 0,
                  tzinfo = london_tz).strftime(f)

'2018-05-12 12:16:00 LMT-0001'

In [12]:
## as highlighted in the pytz documentation using the tzinfo of
## the datetime.datetime initializer does not always lead to the
## desired outcome, such as with the London example
## according to the pytz documentation, this method does lead to
## the desired results in time zones without daylight savings

This is important, such as when you are calculating time deltas. The first example of
the following three is the gotcha:


In [13]:
# generally you want to store data in UTC and convert only when
# generating human-readable output
# you can also do date arithmetic with time zones
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0,
                           tzinfo = london_tz)

In [14]:
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0,
                           tzinfo = western)

In [15]:
event2 - event1

datetime.timedelta(days=1, seconds=17520)

In [16]:
# this yields the wrong time delta because the time zones
# havent been labelled properly

In [17]:
event1 = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))

In [18]:
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [20]:
event1 = london_tz.localize(
            (datetime.datetime(2018, 5, 12, 12, 15, 0))). \
            astimezone(datetime.timezone.utc)

In [21]:
event2 = western.localize(
            (datetime.datetime(2018, 5, 12, 12, 15, 0))). \
            astimezone(datetime.timezone.utc)

In [22]:
event2 - event1

datetime.timedelta(seconds=28800)

`pytz`  provides a list of common time zones and time zones by country, both of which
can be handy references:

In [24]:
pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

A particularly hairy issue is the matter of daylight savings. Certain human-readable
times exist twice (falling behind in the autumn), while others do not exist at all (skip
ahead in the spring):

In [25]:
## time zones
ambig_time = western.localize(
                datetime.datetime(2002, 10, 27, 1, 30, 00)). \
                astimezone(datetime.timezone.utc)

In [27]:
ambig_time_earlier = ambig_time - datetime.timedelta(hours = 1)
ambig_time_earlier

datetime.datetime(2002, 10, 27, 8, 30, tzinfo=datetime.timezone.utc)

In [28]:
ambig_time_later = ambig_time + datetime.timedelta(hours = 1)
ambig_time_later

datetime.datetime(2002, 10, 27, 10, 30, tzinfo=datetime.timezone.utc)

In [29]:
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [30]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [31]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [32]:
# last two are identical!!!
# in this case you need to use is_dst to indicate whether daylight
# saving is in effect


In [33]:
ambig_time = western.localize(
    datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True). \
    astimezone(datetime.timezone.utc)

In [38]:
ambig_time_earlier = ambig_time - datetime.timedelta(hours = 1)
ambig_time_later = ambig_time + datetime.timedelta(hours = 1)

In [39]:
ambig_time_earlier.astimezone(western)


datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [40]:
ambig_time.astimezone(western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [41]:
ambig_time_later.astimezone(western)


datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)